In [1]:
import numpy as np, pandas as pd
import glob
import os 
import json
import sys
import numpy
from math import log, log10
from gdpyc import GasMap, DustMap
from astropy.coordinates import SkyCoord
from astropy import units as u

version = '05012024'

df = pd.read_csv(f'/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_{version}_GammaCan.csv')

df = df[df['FGL'].str[-1]!='c']

df['GLON'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.l.degree
df['GLAT'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.b.degree

df['HR_ms'] = (df['Fcsc_m']-df['Fcsc_s'])/(df['Fcsc_m']+df['Fcsc_s'])
df['HR_hm'] = (df['Fcsc_h']-df['Fcsc_m'])/(df['Fcsc_h']+df['Fcsc_m'])
df = df.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df['name_nospace'] = df['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))


df_alls = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/Classifications/data/Individual_sources/FGL_11152023_GammaCan_alls.csv')

# light curve >50 counts
# spectrum > 100 counts
df_select3 = df 

# df_select3 = df[df['X_name'].isin(df_alls.loc[df_alls['src_cnts_aper90_b']>=50, 'name'].unique())].reset_index(drop=True)

# df_select3 = df_select3[~df_select3['X_name'].isin(['2CXO J174457.8-290509', '2CXO J174507.0-290357','2CXO J174506.8-290537','2CXO J174506.0-290512'])].reset_index(drop=True)
# print(df_select3)
# ,'2CXO J111459.1-611707','2CXO J150234.5+015205','2CXO J073717.0+653557'
coords = SkyCoord(df_select3['ra'], df_select3['dec'], unit='deg')
df_select3['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_select3['nH_from_AV'] = 2.21 * 3.1 * df_select3['ebv'] * 0.1
df_select3['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

print(df_select3[['FGL','X_name','Class','CT','name','name_nospace','ra', 'dec','flux_aper90_ave_b','var_intra_prob',
       'var_inter_prob','nH']])

              FGL                 X_name    Class         CT  \
0    J0859.3-4342  2CXO J085927.0-434528       CV   1.758832   
1    J1725.1-3408  2CXO J172508.8-341112      YSO   0.630623   
2    J0859.2-4729  2CXO J085905.6-473041      YSO   4.232714   
3    J1742.5-2833  2CXO J174216.9-283707       CV   1.401071   
4    J1743.8-3143  2CXO J174347.2-314025  LM-STAR   3.131282   
..            ...                    ...      ...        ...   
112  J0442.8+3609  2CXO J044309.2+360856      AGN  34.009354   
113  J1744.9-2905  2CXO J174507.0-290357       NS   2.350001   
114  J0737.4+6535  2CXO J073726.0+653656      AGN  17.626258   
115  J1115.1-6118  2CXO J111513.6-611657       NS   3.037565   
116  J1115.1-6118  2CXO J111459.1-611707       NS   2.186374   

                        name          name_nospace          ra        dec  \
0    2CXO J085927.0-434528-1  2CXOJ085927.0-434528  134.862609 -43.757805   
1    2CXO J172508.8-341112-1  2CXOJ172508.8-341112  261.286915 -34.186814   


In [2]:
df.columns

Index(['P_AGN', 'P_CV', 'P_HM-STAR', 'P_HMXB', 'P_LM-STAR', 'P_LMXB', 'P_NS',
       'P_YSO', 'e_P_AGN', 'e_P_CV', 'e_P_HM-STAR', 'e_P_HMXB', 'e_P_LM-STAR',
       'e_P_LMXB', 'e_P_NS', 'e_P_YSO', 'Class', 'Class_prob', 'Class_prob_e',
       'name', 'CT', 'X_name', 'FGL', 'ra', 'dec', 'r0', 'significance',
       'Fcsc_s', 'e_Fcsc_s', 'Fcsc_m', 'e_Fcsc_m', 'Fcsc_h', 'e_Fcsc_h',
       'flux_aper90_ave_b', 'e_flux_aper90_ave_b', 'var_intra_prob',
       'var_inter_prob', 'MW_MW', 'MW_counts', 'match_flag', 'p_any', 'p_i',
       'Gmag', 'BPmag', 'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag', 'RPlx', 'PM',
       'epsi', 'sepsi', 'ruwe', 'rgeo', 'Jmag', 'Hmag', 'Kmag', 'e_Jmag',
       'e_Hmag', 'e_Kmag', 'W1mag', 'W2mag', 'W3mag', 'e_W1mag', 'e_W2mag',
       'e_W3mag', 'r_hat', 'X_PA', 'G_ra', 'G_dec', 'G_aF', 'G_bF', 'G_PA',
       'sep', 'Fb', 'HR_hms', 'popup', 'shape', 'color', 'GLON', 'GLAT',
       'HR_ms', 'HR_hm', 'name_nospace', 'ebv', 'nH_from_AV', 'nH'],
      dtype='object')

In [3]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ174457.8-290509':
    #     continue
    #print(src)
    # if index >40:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    try:
        dir_abs = f'/Users/huiyang/Research/GitHub/MUWCLASS_4FGL-DR4/bxa_results/{src}/'

        
        # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

        prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
        models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

        best = max(models, key=models.__getitem__)
        Zbest = models[best]
        for m in models: models[m] -= Zbest
        Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
        limit = 30 # for example, Jeffreys scale for the Bayes factor

        # print()
        # print('Model comparison')
        # print('****************')
        # print()



        for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
            df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
            # print(df_src_sum.columns)
            src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
            Zrel = models[m]
            src_stat['log10Z_'+md] = Zrel / log(10)
            if Zrel == 0:
                src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
        df_sr = pd.DataFrame(data=src_stat, index=[0])
        df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
        # for md in ['powerlaw','mekal','bb']:

        print('  <tr>')
        print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
        print(f"   <td>{src_stat['FGL']}</td>")
        print(f"   <td>{src_stat['l']:.7f}</td>")
        print(f"   <td>{src_stat['b']:.7f}</td>")
        print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        print(f"   <td>{src_stat['CT']:.2f}</td>")
        print(f"   <td>{src_stat['fb']:.2e}</td>")
        print(f"   <td>{src_stat['var_inter']:.2f}</td>")
        print(f'   <td><a class="image-link" href="./images/{src}_lc.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
        print(f"   <td>{src_stat['nobs']:.1f}</td>")
        print(f"   <td>{src_stat['ncounts']:.1f}</td>")
        print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}+{src_stat["pl_u"]-src_stat["pl"]:.2f}-{src_stat["pl"]-src_stat["pl_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}<sup>+{src_stat["pl_u"]-src_stat["pl"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{src_stat["pl"]-src_stat["pl_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}<sup>+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}<sup>+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a></td>')
        print(f"   <td>{src_stat['nH']:.2f}+{src_stat['nH_u']-src_stat['nH']:.2f}-{src_stat['nH']-src_stat['nH_l']:.2}</td>")
        # print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
        print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
        print('  </tr>')

    except:
        print('  <tr>')
        print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
        print(f"   <td>{src_stat['FGL']}</td>")
        print(f"   <td>{src_stat['l']:.7f}</td>")
        print(f"   <td>{src_stat['b']:.7f}</td>")
        print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        print(f"   <td>{src_stat['CT']:.2f}</td>")
        print(f"   <td>{src_stat['fb']:.2e}</td>")
        print(f"   <td>{src_stat['var_inter']:.2f}</td>")
        print(f'   <td>{src_stat["var_intra"]:.2f}</td>')
        print(f"   <td>{src_stat['nobs']:.1f}</td>")
        print(f"   <td>{src_stat['ncounts']:.1f}</td>")
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
        print('  </tr>')

    

  <tr>
   <td><a href='./srcs/2CXOJ085927.0-434528.html'>J085927.0-434528</a></td>
   <td>J0859.3-4342</td>
   <td>265.1473173</td>
   <td>1.4499867</td>
   <td><a class="image-link" href="./images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>
   <td>1.76</td>
   <td>1.59e-12</td>
   <td>nan</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_lc.png" target="_blank">1.00</a></td>
   <td>1.0</td>
   <td>6082.5</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_spectrum.png" target="_blank">1.63+0.06-0.054</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_corner.pdf" target="_blank">-10.45</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_spectrum.png" target="_blank">7.82+0.81-0.66</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_corner.pdf" target="_blank">0.00</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_bb_spectrum.png" tar

In [6]:
df_srcs.to_csv('FGL_11152023_GammaCan_summary.csv',index=False)

In [104]:
len(df_select3)

130

In [5]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ172508.8-341112':
    #     continue
    #print(src)
    # if index >0:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')
    morelines = 2705
    try:
        dir_abs = f'/Users/huiyang/Research/GitHub/FGL_spectral_summary/bxa_results/{src}/'

        
        # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

        prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
        models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

        best = max(models, key=models.__getitem__)
        Zbest = models[best]
        for m in models: models[m] -= Zbest
        Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
        limit = 30 # for example, Jeffreys scale for the Bayes factor

        # print()
        # print('Model comparison')
        # print('****************')
        # print()



        for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
            df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
            # print(df_src_sum.columns)
            src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
            Zrel = models[m]
            src_stat['log10Z_'+md] = Zrel / log(10)
            if Zrel == 0:
                src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
        df_sr = pd.DataFrame(data=src_stat, index=[0])
        df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
        # for md in ['powerlaw','mekal','bb']:

        temp_html = open('./srcs/template.html','r')

        lines = temp_html.readlines()

        lines[29] = lines[29].replace('2CXO J085927.0-434528', src_name)
        lines[101] = lines[101].replace('134.862609 -43.757805', f"{df_src['ra']}, {df_src['dec']}")

        lines[224+morelines] = lines[224+morelines].replace('J085927.0-434528',src)
        lines[225+morelines] = lines[225+morelines].replace('J0859.3-4342',df_src['FGL'])
        lines[226+morelines] = lines[226+morelines].replace('265.1473173',f"{src_stat['l']:.7f}")
        lines[227+morelines] = lines[227+morelines].replace('1.4499867',f"{src_stat['b']:.7f}")
        # lines[228+morelines] = lines[228+morelines].replace('<td><a class="image-link" href="../images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>',f'<td><a class="image-link" href="../images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        lines[228+morelines] = lines[228+morelines].replace('<td><a class="image-link" href="../images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>',f'<td>{src_stat["Class"]}</td>')
        lines[229+morelines] = lines[229+morelines].replace('0.658442',f"{df_src['Class_prob']:.3f}")
        lines[230+morelines] = lines[230+morelines].replace('0.108274',f"{df_src['Class_prob_e']:.3f}")
        lines[231+morelines] = lines[231+morelines].replace('2.45',f"{df_src['CT']:.2f}")
        
        lines[255+morelines] = lines[255+morelines].replace('1.59e-12',f"{df_src['flux_aper90_ave_b']:.2e}")
        lines[256+morelines] = lines[256+morelines].replace('8.90e-15',f"{df_src['Fcsc_s']:.2e}")
        lines[257+morelines] = lines[257+morelines].replace('1.10e-13',f"{df_src['Fcsc_m']:.2e}")
        lines[258+morelines] = lines[258+morelines].replace('1.47e-12',f"{df_src['Fcsc_h']:.2e}")
        lines[259+morelines] = lines[259+morelines].replace('0.84965',f"{df_src['HR_hms']:.2f}")
        lines[260+morelines] = lines[260+morelines].replace('0.850961',f"{df_src['HR_ms']:.2f}")
        lines[261+morelines] = lines[261+morelines].replace('0.86007',f"{df_src['HR_hm']:.2f}")
        lines[262+morelines] = lines[262+morelines].replace('nan',f"{df_src['var_inter_prob']:.2f}")
        lines[263+morelines] = lines[263+morelines].replace('<a class="image-link" href="../images/2CXOJ085927.0-434528_lc_500s.png" target="_blank">1.00</a>',f'<a class="image-link" href="../images/{src}_lc.png" target="_blank">{src_stat["var_intra"]:.2f}</a>')
        lines[264+morelines] = lines[264+morelines].replace('1.0',f"{src_stat['nobs']:.1f}")
        lines[265+morelines] = lines[265+morelines].replace('6082.5',f"{src_stat['ncounts']:.0f}")

        lines[292+morelines] = lines[292+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_pl_spectrum.png" target="_blank">3.61</a>',f'<a class="image-link" href="../images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}</a>')
        lines[293+morelines] = lines[293+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_pl_corner.pdf" target="_blank">-6.37</a>',f'<a class="image-link" href="../images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a>')
        lines[294+morelines] = lines[294+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_mk_spectrum.png" target="_blank">1.28</a>',f'<a class="image-link" href="../images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}</a>')
        lines[295+morelines] = lines[295+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_mk_corner.pdf" target="_blank">0.00</a>',f'<a class="image-link" href="../images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a>')
        lines[296+morelines] = lines[296+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_bb_spectrum.png" target="_blank">0.59</a>',f'<a class="image-link" href="../images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}</a>')
        lines[297+morelines] = lines[297+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_bb_corner.pdf" target="_blank">-47.39</a>',f'<a class="image-link" href="../images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a>')
        lines[298+morelines] = lines[298+morelines].replace('22.42',f"{src_stat['nH']:.2f}")
        lines[299+morelines] = lines[299+morelines].replace('21.81',f"{src_stat['nH_LAB']:.2f}")

        lines[322+morelines] = lines[322+morelines].replace('18.130289',f"{df_src['Gmag']:.3f}")
        lines[323+morelines] = lines[323+morelines].replace('20.997097',f"{df_src['BPmag']:.3f}")
        lines[324+morelines] = lines[324+morelines].replace('16.169699',f"{df_src['RPmag']:.3f}")
        lines[325+morelines] = lines[325+morelines].replace('<td></td>',f"<td>{df_src['Jmag']:.3f}</td>")
        lines[326+morelines] = lines[326+morelines].replace('<td></td>',f"<td>{df_src['Hmag']:.3f}</td>")
        lines[327+morelines] = lines[327+morelines].replace('<td></td>',f"<td>{df_src['Kmag']:.3f}</td>")
        lines[328+morelines] = lines[328+morelines].replace('<td></td>',f"<td>{df_src['W1mag']:.3f}</td>")
        lines[329+morelines] = lines[329+morelines].replace('<td></td>',f"<td>{df_src['W2mag']:.3f}</td>")
        lines[330+morelines] = lines[330+morelines].replace('1580.09277',f"{df_src['rgeo']:.1f}")
        lines[331+morelines] = lines[331+morelines].replace('<td></td>',f"<td>{df_src['RPlx']:.3f}</td>")
        lines[332+morelines] = lines[332+morelines].replace('0.009257',f"{df_src['PM']:.5f}")


        src_html = open(f'./srcs/{src}.html','w')
        
        src_html.writelines((lines))
        src_html.close()

    except:
        df_sr = pd.DataFrame(data=src_stat, index=[0])
        temp_html = open('./srcs/template.html','r')

        lines = temp_html.readlines()

        lines[29] = lines[29].replace('2CXO J085927.0-434528', src_name)
        lines[101] = lines[101].replace('134.862609 -43.757805', f"{df_src['ra']}, {df_src['dec']}")

        lines[224+morelines] = lines[224+morelines].replace('J085927.0-434528',src)
        lines[225+morelines] = lines[225+morelines].replace('J0859.3-4342',df_src['FGL'])
        lines[226+morelines] = lines[226+morelines].replace('265.1473173',f"{src_stat['l']:.7f}")
        lines[227+morelines] = lines[227+morelines].replace('1.4499867',f"{src_stat['b']:.7f}")
        # lines[228+morelines] = lines[228+morelines].replace('<td><a class="image-link" href="../images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>',f'<td><a class="image-link" href="../images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        lines[228+morelines] = lines[228+morelines].replace('<td><a class="image-link" href="../images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>',f'<td>{src_stat["Class"]}</td>')
        lines[229+morelines] = lines[229+morelines].replace('0.658442',f"{df_src['Class_prob']:.3f}")
        lines[230+morelines] = lines[230+morelines].replace('0.108274',f"{df_src['Class_prob_e']:.3f}")
        lines[231+morelines] = lines[231+morelines].replace('2.45',f"{df_src['CT']:.2f}")
        
        lines[255+morelines] = lines[255+morelines].replace('1.59e-12',f"{df_src['flux_aper90_ave_b']:.2e}")
        lines[256+morelines] = lines[256+morelines].replace('8.90e-15',f"{df_src['Fcsc_s']:.2e}")
        lines[257+morelines] = lines[257+morelines].replace('1.10e-13',f"{df_src['Fcsc_m']:.2e}")
        lines[258+morelines] = lines[258+morelines].replace('1.47e-12',f"{df_src['Fcsc_h']:.2e}")
        lines[259+morelines] = lines[259+morelines].replace('0.84965',f"{df_src['HR_hms']:.2f}")
        lines[260+morelines] = lines[260+morelines].replace('0.850961',f"{df_src['HR_ms']:.2f}")
        lines[261+morelines] = lines[261+morelines].replace('0.86007',f"{df_src['HR_hm']:.2f}")
        lines[262+morelines] = lines[262+morelines].replace('nan',f"{df_src['var_inter_prob']:.2f}")
        lines[263+morelines] = lines[263+morelines].replace('<a class="image-link" href="../images/2CXOJ085927.0-434528_lc_500s.png" target="_blank">1.00</a>',f'<a class="image-link" href="../images/{src}_lc.png" target="_blank">{src_stat["var_intra"]:.2f}</a>')
        lines[264+morelines] = lines[264+morelines].replace('1.0',f"{src_stat['nobs']:.1f}")
        lines[265+morelines] = lines[265+morelines].replace('6082.5',f"{src_stat['ncounts']:.0f}")

        lines[292+morelines] = lines[292+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_pl_spectrum.png" target="_blank">3.61</a>','')
        lines[293+morelines] = lines[293+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_pl_corner.pdf" target="_blank">-6.37</a>','')
        lines[294+morelines] = lines[294+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_mk_spectrum.png" target="_blank">1.28</a>','')
        lines[295+morelines] = lines[295+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_mk_corner.pdf" target="_blank">0.00</a>','')
        lines[296+morelines] = lines[296+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_bb_spectrum.png" target="_blank">0.59</a>','')
        lines[297+morelines] = lines[297+morelines].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_bb_corner.pdf" target="_blank">-47.39</a>','')
        lines[298+morelines] = lines[298+morelines].replace('22.42','')
        lines[299+morelines] = lines[299+morelines].replace('21.81',f"{src_stat['nH_LAB']:.2f}")

        lines[322+morelines] = lines[322+morelines].replace('18.130289',f"{df_src['Gmag']:.3f}")
        lines[323+morelines] = lines[323+morelines].replace('20.997097',f"{df_src['BPmag']:.3f}")
        lines[324+morelines] = lines[324+morelines].replace('16.169699',f"{df_src['RPmag']:.3f}")
        lines[325+morelines] = lines[325+morelines].replace('<td></td>',f"<td>{df_src['Jmag']:.3f}</td>")
        lines[326+morelines] = lines[326+morelines].replace('<td></td>',f"<td>{df_src['Hmag']:.3f}</td>")
        lines[327+morelines] = lines[327+morelines].replace('<td></td>',f"<td>{df_src['Kmag']:.3f}</td>")
        lines[328+morelines] = lines[328+morelines].replace('<td></td>',f"<td>{df_src['W1mag']:.3f}</td>")
        lines[329+morelines] = lines[329+morelines].replace('<td></td>',f"<td>{df_src['W2mag']:.3f}</td>")
        lines[330+morelines] = lines[330+morelines].replace('1580.09277',f"{df_src['rgeo']:.1f}")
        lines[331+morelines] = lines[331+morelines].replace('<td></td>',f"<td>{df_src['RPlx']:.3f}</td>")
        lines[332+morelines] = lines[332+morelines].replace('0.009257',f"{df_src['PM']:.5f}")


        src_html = open(f'./srcs/{src}.html','w')
        
        src_html.writelines((lines))
        src_html.close()

    # count = 0
    # for line in lines:
    #     count +=1
    #     if count == 66:
    #         print(line)

    # print('  <tr>')
    # print(f"   <td>{src_stat['name']}</td>")
    # print(f"   <td>{src_stat['FGL']}</td>")
    # print(f"   <td>{src_stat['l']:.7f}</td>")
    # print(f"   <td>{src_stat['b']:.7f}</td>")
    # print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
    # print(f"   <td>{src_stat['CT']:.2f}</td>")
    # print(f"   <td>{src_stat['fb']:.2e}</td>")
    # print(f"   <td>{src_stat['var_inter']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_lc_500s.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
    # print(f"   <td>{src_stat['nobs']:.1f}</td>")
    # print(f"   <td>{src_stat['ncounts']:.1f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}<sup>+{src_stat["pl_u"]-src_stat["pl"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{src_stat["pl"]-src_stat["pl_l"]:.2}</sub></a></td>')
    # # print(f"   <td>{src_stat['log10Z_pl']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a></td>')
    # print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}<sup>+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</sub></a></td>')
    # print(f'   <td><a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a></td>')
    # # print(f"   <td>{src_stat['log10Z_mk']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}<sup>+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</sub></a></td>')
    # # print(f"   <td>{src_stat['log10Z_bb']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a></td>')
    # print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
    # print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
    # print('  </tr>')


    

In [28]:
df_select3.columns

df_select3.loc[df_select3['X_name']=='2CXO J085927.0-434528', ['FGL','X_name',#'ra','dec','GLON', 'GLAT','Class','Class_prob','Class_prob_e','CT',]]#
# 'flux_aper90_ave_b','Fcsc_s','Fcsc_m','Fcsc_h','var_intra_prob','var_inter_prob','HR_hms','HR_ms','HR_hm']]#
 'Gmag', 'BPmag', 'RPmag','Jmag', 'Hmag', 'Kmag','W1mag', 'W2mag','rgeo','PM',]]

,FGL,X_name,Gmag,BPmag,RPmag,Jmag,Hmag,Kmag,W1mag,W2mag,rgeo,PM
0,J0859.3-4342,2CXO J085927.0-434528,18.130289,20.997097,16.169699,NaN,NaN,NaN,NaN,NaN,1580.09277,0.009257


In [113]:
import math

# Given values
flux = 7.4e-14 #7.15e-13  # erg/s/cm^2
distance_kpc = 0.716 # kpc

# Convert distance to centimeters
distance_cm = distance_kpc * 3.086e21  # 1 parsec = 3.086e18 cm

# Calculate isotropic luminosity
luminosity = 4 * math.pi * (distance_cm**2) * flux

print(f"The isotropic luminosity is approximately {luminosity:.2e} erg/s.")


The isotropic luminosity is approximately 4.54e+30 erg/s.


In [22]:
from os import path

def CSCview_conesearch(field_name, ra, dec, radius,query_dir,engine='curl',csc_version='2.0',adql_version='csc_query_cnt_template',num_header=154,rerun=False,data_dir='./'):
    
    if not (path.exists(f'{query_dir}/{field_name}_{engine}.txt')) or rerun==True: 

       
        ra_low  = ra - radius/(60.*np.cos(dec*np.pi/180.))
        ra_upp  = ra + radius/(60.*np.cos(dec*np.pi/180.))
        dec_low = dec - radius/60
        dec_upp = dec + radius/60
        rad_cone = radius
        
        
        f = open(f'{query_dir}/template/{adql_version}.adql', "r")
        adql = f.readline()
        ra_temp = '266.599396'
        dec_temp = '-28.87594'
        ra_low_temp = '266.5898794490786'
        ra_upp_temp = '266.60891255092145'
        dec_low_temp = '-28.884273333333333'
        dec_upp_temp = '-28.867606666666667'
        rad_cone_temp = '0.543215'
        #'''
        for [str1, str2] in [[rad_cone, rad_cone_temp], [ra, ra_temp], [dec, dec_temp], [ra_low, ra_low_temp], [ra_upp, ra_upp_temp], [dec_low, dec_low_temp], [dec_upp, dec_upp_temp]]:
            adql = adql.replace(str2, str(str1))
        

        #text_file = open(f'{query_dir}/{field_name}.adql', "w")
        #text_file.write(adql)
        #text_file.close()

        if engine == 'curl':
            text_file = open(f'{query_dir}/{field_name}.adql', "w")
            text_file.write(adql)
            text_file.close()

            os.system("curl -o "+query_dir+'/'+field_name+"_"+engine+".txt \
                --form version="+csc_version+"  \
                --form query=@"+query_dir+'/'+field_name+".adql \
                http://cda.cfa.harvard.edu/csccli/getProperties")
            
        elif engine == 'wget':
            text_file = open(f'{query_dir}/{field_name}_wget.adql', "w")
            text_file.write('http://cda.cfa.harvard.edu/csccli/getProperties?query='+adql)
            text_file.close()

            os.system("wget -O "+query_dir+'/'+field_name+"_"+engine+".txt -i "+query_dir+'/'+field_name+"_wget.adql")
            

        #df = pd.read_csv(f'{query_dir}/{field_name}.txt', header=15, sep='\t')

    # df = pd.read_csv(f'{query_dir}/{field_name}_{engine}.txt', header=num_header, sep='\t')
    # df['name'] = df['name'].str.lstrip()
    

    return df

In [3]:
import warnings
warnings.filterwarnings('ignore')

dfs1 = pd.DataFrame()
dfs2 = pd.DataFrame()
for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ103220.2+573211':
    #     continue
    #print(src)
    # if index !=3:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    query_dir = '/Users/huiyang/Research/GitHub/MUWCLASS_4FGL-DR4/srcs/query'
    # df = CSCview_conesearch(src, ra, dec, 2./60,query_dir,engine='curl',csc_version='current',adql_version='cscview_s2',num_header=154,rerun=False,data_dir='./')
    # df = CSCview_conesearch(src, ra, dec, 2./60,query_dir,engine='wget',csc_version='2.0',adql_version='cscview_s2',num_header=154,rerun=False,data_dir='./')
    df1 = pd.read_csv(f'{query_dir}/{src}_curl.txt', header=214, sep='\t')
    dfs1 = pd.concat([dfs1, df1], ignore_index=True, sort=False)
    df2 = pd.read_csv(f'{query_dir}/{src}_wget.txt', header=214, sep='\t')
    dfs2 = pd.concat([dfs2, df2], ignore_index=True, sort=False)
    if len(df1)==0:
        print('0',src)
    elif len(df1['name'].unique())>1:
        print('1',src)
    if len(df2)==0:
        print('2!!!!!',src)
    elif len(df2['name'].unique())>1:
        print('3',src)
    # if len(df1)>1:
    #     print(src)

3 2CXOJ085905.6-473041
0 2CXOJ174216.9-283707
0 2CXOJ174347.2-314025
0 2CXOJ173411.3-293117
0 2CXOJ174042.0-280724
0 2CXOJ173657.4-285630
0 2CXOJ173927.6-272112
0 2CXOJ175039.6-302056
0 2CXOJ175020.7-302652
0 2CXOJ174036.5-280840
0 2CXOJ175709.5-272532
0 2CXOJ174237.6-283726
0 2CXOJ175147.9-301046
1 2CXOJ085905.5-473036
3 2CXOJ085905.5-473036
0 2CXOJ174457.8-290509
0 2CXOJ174548.1-314318
0 2CXOJ173527.5-294516
0 2CXOJ175734.9-273654
0 2CXOJ174249.7-283552
1 2CXOJ102557.9+124109
3 2CXOJ102557.9+124109
0 2CXOJ174507.0-290357
1 2CXOJ111510.3-611640
3 2CXOJ111510.3-611640
0 2CXOJ174506.8-290537
0 2CXOJ174506.0-290512


In [4]:
len(dfs1), len(dfs2)

dfs1['version'] = 'current'
dfs2['version'] = 'cscv2.0'

dfs = pd.concat([dfs2,dfs1], ignore_index=True,sort=False)
dfs['name'] = dfs['name'].str.lstrip()
dfs = dfs[dfs['name'].isin(df_select3['X_name'])].reset_index(drop=True).sort_values(by=['name','version']).reset_index(drop=True)

dfs.columns[:100]


Index(['separation', 'name', 'ra', 'dec', 'gal_l', 'gal_b', 'err_ellipse_r0',
       'err_ellipse_r1', 'err_ellipse_ang', 'significance', 'likelihood',
       'likelihood_class', 'conf_flag', 'dither_warning_flag', 'extent_flag',
       'pileup_flag', 'sat_src_flag', 'streak_src_flag', 'var_flag',
       'var_inter_hard_flag', 'man_add_flag', 'man_inc_flag', 'man_match_flag',
       'man_pos_flag', 'man_reg_flag', 'flux_aper90_b', 'flux_aper90_lolim_b',
       'flux_aper90_hilim_b', 'flux_aper90_h', 'flux_aper90_lolim_h',
       'flux_aper90_hilim_h', 'flux_aper90_m', 'flux_aper90_lolim_m',
       'flux_aper90_hilim_m', 'flux_aper90_s', 'flux_aper90_lolim_s',
       'flux_aper90_hilim_s', 'flux_aper90_u', 'flux_aper90_lolim_u',
       'flux_aper90_hilim_u', 'phot_nsrcs', 'nh_gal', 'hard_hm',
       'hard_hm_lolim', 'hard_hm_hilim', 'var_inter_hard_prob_hm',
       'var_inter_hard_sigma_hm', 'hard_hs', 'hard_hs_lolim', 'hard_hs_hilim',
       'var_inter_hard_prob_hs', 'var_inter_hard_si

In [8]:
dfs.loc[dfs['name']=='2CXO J102557.9+124109', ['name','ra','dec','err_ellipse_r0','extent_flag','obsid','gti_elapse','theta','powlaw_gamma', 'powlaw_gamma_lolim',
       'powlaw_gamma_hilim','flux_aper90_b.1','phot_nsrcs.1','livetime','version']]

,name,ra,dec,err_ellipse_r0,extent_flag,obsid,gti_elapse,theta,powlaw_gamma,powlaw_gamma_lolim,powlaw_gamma_hilim,flux_aper90_b.1,phot_nsrcs.1,livetime,version
146,2CXO J102557.9+124109,10 25 57.98,+12 41 09.46,0.74,TRUE,13375,9053.977,3.192,2.530,2.159,2.983,5.749e-14,1,8935.7,cscv2.0
147,2CXO J102557.9+124109,10 25 57.95,+12 41 09.20,0.37,TRUE,13375,9053.977,3.192,2.530,2.159,2.983,5.749e-14,1,8935.7,current


In [13]:
dfs1['name']

# df_select3

0       2CXO J085927.0-434528
1       2CXO J172508.8-341112
2       2CXO J172508.8-341112
3       2CXO J172508.8-341112
4       2CXO J085905.6-473041
                ...          
286     2CXO J111459.1-611707
287     2CXO J111459.1-611707
288     2CXO J111459.1-611707
289     2CXO J111459.1-611707
290     2CXO J111459.1-611707
Name: name, Length: 291, dtype: object

## bright extended sources that we removed from analysis

In [99]:
df_extend = pd.DataFrame(data={'name':['2CXO J125724.4+272952','2CXO J125711.9+274216','2CXO J223704.7+184056','2CXO J205855.7+440337','2CXO J073706.8+653636','2CXO J110807.9+091722','2CXO J111636.5+181552','2CXO J161532.9-603954', '2CXO J170000.0-455935','2CXO J170043.7-455523','2CXO J170142.3-462504','2CXO J172051.8-371037'],
                                'ra':[194.351708333333,        194.29987499999996,   339.269833333333,  314.73212499999994, 114.27841666666667,167.0329583333333, 169.15237499999998,243.8872083333333, 255.00016666666662, 255.18216666666663, 255.42662499999994, 260.21583333333325],
                                'dec':[27.49791388888889,      27.70468888888889,    18.68241944444444,  44.060297222222, 65.6100083333333,9.289552777777777, 18.264672222222224, -60.66526944444444, -45.99319444444445, -45.92326944444444, -46.4178138888888, -37.17705555555555]})

df_extend = df_extend[~df_extend['name'].isin(['2CXO J073706.8+653636'])].sort_values(by=['name']).reset_index(drop=True)

# '2CXO J073706.8+653636': too faint, 

df_CSC_all = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_CSCv2/old_files/CSCv2_all_MW_classprob_XCLASS.csv')

df_fgl_unas = pd.read_csv('/Users/huiyang/Research/GitHub/FGL_Class/4fgl_dr4/fgl_v32_4FGLUS_CSC2.csv')

df_CSC = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_XMM/codes/MW_match/data/4FGL-DR4_nway_match/FGLDR4_MW_all.csv')# FGLDR4_MW.csv')#FGLDR4_MW_all.csv')
print(len(df_CSC))


47337


In [79]:
df_extend.sort_values(by='name')

,name,ra,dec
0,2CXO J073706.8+653636,114.278417,65.610008


In [80]:
df_CSC.columns

# 2CXO J125724.4+272952
# df_CSC_all.loc[df_CSC_all['name'].isin(df_extend['name']), ['name','Signif.']]
df_CSC.loc[df_CSC['name'].isin(df_extend['name']), ['name','GAIA_E_BP_RP_','ALLWISE_ex','match_flag','p_any','p_i']]

,name,GAIA_E_BP_RP_,ALLWISE_ex,match_flag,p_any,p_i
43210,2CXO J073706.8+653636,NaN,NaN,0,0.987885,0.000000
43211,2CXO J073706.8+653636,NaN,NaN,2,0.987885,0.424541
43212,2CXO J073706.8+653636,NaN,NaN,1,0.987885,0.575459


In [107]:


df_extends1 = pd.DataFrame()
df_extends2 = pd.DataFrame()
for index, df_src in df_extend.iterrows():# ['name_nospace'].values:
    
    src = df_src['name'].replace(' ','') #strip() 
    src_short = src[4:]
    # if src != '2CXOJ103220.2+573211':
    #     continue
    #print(src)
    # if index !=3:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec  = df_src['ra'], df_src['dec']
    print(src, ra, dec)
    query_dir = '/Users/huiyang/Research/GitHub/MUWCLASS_4FGL-DR4/srcs/query'
    # df = CSCview_conesearch(src, ra, dec, 2./60,query_dir,engine='curl',csc_version='current',adql_version='cscview_s2',num_header=166,rerun=True,data_dir='./')
    # df = CSCview_conesearch(src, ra, dec, 2./60,query_dir,engine='wget',csc_version='2.0',adql_version='cscview_s2',num_header=166,rerun=True,data_dir='./')
    df1 = pd.read_csv(f'{query_dir}/{src}_curl.txt', header=226, sep='\t')
    df_extends1 = pd.concat([df_extends1, df1], ignore_index=True, sort=False)
    df2 = pd.read_csv(f'{query_dir}/{src}_wget.txt', header=226, sep='\t')
    df_extends2 = pd.concat([df_extends2, df2], ignore_index=True, sort=False)
    # if len(df1)==0:
    #     print('0',src)
    # elif len(df1['name'].unique())>1:
    #     print('1',src)
    # if len(df2)==0:
    #     print('2!!!!!',src)
    # elif len(df2['name'].unique())>1:
       

len(dfs1), len(dfs2)

df_extends1['version'] = 'current'
df_extends2['version'] = 'cscv2.0'
print(len(df_extends1['name'].unique()), len(df_extends2['name'].unique()))
df_extends = df_extends2 #pd.concat([df_extends1,df_extends2], ignore_index=True,sort=False)
df_extends['name'] = df_extends['name'].str.lstrip()
# df_extends = df_extends[dfs['name'].isin(df_select3['X_name'])].reset_index(drop=True).sort_values(by=['name','version']).reset_index(drop=True)

df_extends_master = df_extends.drop_duplicates(subset=['name']).reset_index(drop=True)

len(df_extends['name'].unique())

df_extends_master.to_csv('./df_extends_master.csv',index=False)
df_extends.to_csv('./df_extends.csv',index=False)
df_extends.columns[:100]

2CXOJ110807.9+091722 167.0329583333333 9.289552777777777
2CXOJ111636.5+181552 169.15237499999998 18.264672222222224
2CXOJ125711.9+274216 194.29987499999996 27.70468888888889
2CXOJ125724.4+272952 194.351708333333 27.49791388888889
2CXOJ161532.9-603954 243.8872083333333 -60.66526944444444
2CXOJ170000.0-455935 255.00016666666662 -45.99319444444445
2CXOJ170043.7-455523 255.18216666666663 -45.92326944444444
2CXOJ170142.3-462504 255.42662499999994 -46.4178138888888
2CXOJ172051.8-371037 260.21583333333325 -37.17705555555555
2CXOJ205855.7+440337 314.73212499999994 44.060297222222
2CXOJ223704.7+184056 339.269833333333 18.68241944444444
10 11


Index(['separation', 'name', 'ra', 'dec', 'gal_l', 'gal_b', 'err_ellipse_r0',
       'err_ellipse_r1', 'err_ellipse_ang', 'significance', 'likelihood',
       'likelihood_class', 'conf_flag', 'dither_warning_flag', 'extent_flag',
       'pileup_flag', 'sat_src_flag', 'streak_src_flag', 'var_flag',
       'var_inter_hard_flag', 'man_add_flag', 'man_inc_flag', 'man_match_flag',
       'man_pos_flag', 'man_reg_flag', 'flux_aper_b', 'flux_aper_lolim_b',
       'flux_aper_hilim_b', 'flux_aper_h', 'flux_aper_lolim_h',
       'flux_aper_hilim_h', 'flux_aper_m', 'flux_aper_lolim_m',
       'flux_aper_hilim_m', 'flux_aper_s', 'flux_aper_lolim_s',
       'flux_aper_hilim_s', 'flux_aper90_b', 'flux_aper90_lolim_b',
       'flux_aper90_hilim_b', 'flux_aper90_h', 'flux_aper90_lolim_h',
       'flux_aper90_hilim_h', 'flux_aper90_m', 'flux_aper90_lolim_m',
       'flux_aper90_hilim_m', 'flux_aper90_s', 'flux_aper90_lolim_s',
       'flux_aper90_hilim_s', 'flux_aper90_u', 'flux_aper90_lolim_u',
      

In [101]:
df_extends_master[['name','ra','dec','significance','likelihood', 'extent_flag', 'flux_aper90_b','phot_nsrcs', 'nh_gal', 'hard_hm', ]]

,name,ra,dec,significance,likelihood,extent_flag,flux_aper90_b,phot_nsrcs,nh_gal,hard_hm
0,2CXO J110807.9+091722,11 08 07.91,+09 17 22.39,14.54,702.77940,TRUE,1.936000e-13,1,2.75,-0.1249
1,2CXO J111636.5+181552,11 16 36.57,+18 15 52.82,5.58,76.89581,False,1.783000e-14,1,1.45,0.0062
2,2CXO J125711.9+274216,12 57 11.97,+27 42 16.88,12.78,487.97230,False,7.908000e-14,1,0.92,-0.0999
3,2CXO J125724.4+272952,12 57 24.41,+27 29 52.49,9.40,62.52540,TRUE,5.756000e-15,1,0.92,-0.6871
4,2CXO J161532.9-603954,16 15 32.93,-60 39 54.97,11.53,558.28970,TRUE,1.322000e-13,1,20.64,-0.0450
5,2CXO J170000.0-455935,17 00 00.04,-45 59 35.50,16.13,673.49910,TRUE,4.272000e-14,1,89.16,0.7408
6,2CXO J170043.7-455523,17 00 43.72,-45 55 23.77,5.86,84.63325,TRUE,0.000000e+00,1,87.24,-0.2673
7,2CXO J170142.3-462504,17 01 42.39,-46 25 04.13,11.66,314.19100,False,3.492000e-14,1,80.02,-0.2186
8,2CXO J172051.8-371037,17 20 51.80,-37 10 37.40,15.46,1886.60500,False,2.026000e-13,1,173.60,-0.2911
9,2CXO J205855.7+440337,20 58 55.71,+44 03 37.07,26.27,2943.73800,False,3.714000e-13,1,80.74,-0.7695


In [87]:
df_extends_master.loc[0, 'flux_aper90_b']

5.813e-15